1. Import and config 

In [ ]:
import os
import datetime
import cv2

############## user define #############
ImageSize = 32
epoch = 10
netName = 'VGG9ave'
binary = True
marker_type = 'cross'
img_num = 9000
#####################################

DatasetName = 'linetrace_' + str(ImageSize)
ProjectName = 'Project_' + DatasetName + '_' + str(datetime.date.today()) + '_' + netName+ '_epoch' +str(epoch)
ProjectName = ProjectName + ('_imgnum' + str(img_num) + '_marker' + marker_type)

!pip3 install 'cupy-cuda101>=7.7.0,<8.0.0'
!pip3 install chainer

2. Download GUINNESS repository from github

In [ ]:
%cd /content
if (os.path.exists('/content/CUINNESS')==False):
    !git clone https://github.com/knmrtkt/CUINNESS.git
else:
    !git pull
%cd CUINNESS
!git pull

3. Generate line images

Note!!
If you want to modify the line generate algorithm, please modify CUINNESS/gen_line_image.py.
Double click the file from left side file viewer and modify.

In [ ]:
%cd /content
from CUINNESS import gen_line_image
img_gen = gen_line_image.gen_line_image()
img_gen.generate_image(ImageSize, ImageSize, 3, 2, 'linetrace/', 'dataset.csv', img_num, 4, {"straight" : "0", "right" : "1", "left" : "2",})
!rm linetrace/dataset.csv
img_gen.mark_image(ImageSize, ImageSize, 'linetrace', 4, 0, marker_type=marker_type)
%cd /content/CUINNESS
!ls -1 -d ../linetrace/* > list_tmp1.txt
!ls -1 ../linetrace > list_tmp2.txt
!paste -d ' ' list_tmp1.txt list_tmp2.txt > ../CUINNESS/list.txt
!rm list_tmp1.txt list_tmp2.txt

4. Convert line images to datasets for GUINNESS

In [ ]:
%cd /content/CUINNESS
!python3 gen_training_data.py --pathfile list.txt --dataset $DatasetName --size $ImageSize --keepaspect yes

5. Training and evaluation. After that project files are downloaded automatically.

In [ ]:

%cd /content/CUINNESS
!python3 guinness.py $ProjectName $DatasetName $epoch --netName=$netName
%cd $ProjectName
!python3 eval.py --dataset ../$DatasetName --size $ImageSize --testnum 100 > eval_result.txt
%cd ..
ProjectZipPath = '/content/CUINNESS/' + ProjectName + '.zip'
!zip -r $ProjectZipPath /content/CUINNESS/$ProjectName
from google.colab import files
files.download(ProjectZipPath)